In [10]:
import gradio as gr
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown
import requests

e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [32]:

load_dotenv(override=True)
openai_api_key = os.getenv("OpenAI_API_KEY")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
if requests.get("http://localhost:11434").ok:
    ollama_local = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
if openai_api_key:
    openai = OpenAI()
else:
    print("no llm reachable")

In [26]:
def chat_with_local_ollama(message):
    messages = []
    messages = [{"role" : "system", "content" : "you are argumentative assistant, You disagree with everything and respond in a rude manner" }]
    messages.append({"role" : "user", "content" : message})
    response =  ""
    stream = ollama_local.chat.completions.create(model="gpt-oss:20b",messages=messages,stream=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response


In [33]:
def chat_with_gpt(message):
    messages = []
    messages = [{"role" : "system", "content" : "you are argumentative assistant, You disagree with everything and respond in a rude manner" }]
    messages.append({"role" : "user", "content" : message})
    response =  ""
    stream = openai.chat.completions.create(model="gpt-4.1-mini",messages=messages,stream=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [36]:
def call_llms(message,model):
    if model == "gpt":
        result=chat_with_gpt(message)
    if model == "ollama":
        result=chat_with_local_ollama(message)
    yield from result


In [41]:
message_input = gr.Textbox(label="Your message:", info="Enter a message to llm", lines=7)
model_selector = gr.Dropdown(choices=["ollama", "gpt"], label="select a model", value= "gpt")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=call_llms,
    title="LLMs", 
    inputs=[message_input,model_selector], 
    outputs=[message_output], 
    examples=[
        ["Explain the Transformer architecture to a layperson", "gpt"],
        ["Explain the Transformer architecture to an aspiring AI engineer", "ollama"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
